# Домашнее задание #3

<pre>
SELECT 'ФИО: Никифоров Владимир' as FIO;
</pre>

In [1]:
from IPython.display import Image

![FIO](../img/hw2_0.png)

### Оконные функции.

Вывести список пользователей в формате userId, movieId, normed_rating, avg_rating где

- userId, movieId - без изменения
- для каждого пользователя преобразовать рейтинг r в нормированный - normed_rating=(r - r_min)/(r_max - r_min), где r_min и r_max соответственно минимально и максимальное значение рейтинга у данного пользователя
- avg_rating - среднее значение рейтинга у данного пользователя

Вывести первые 30 таких записей

<pre>
-- оконные функции
SELECT userId, movieId,
       case when MAX(rating) OVER (PARTITION BY userId) = MIN(rating) OVER (PARTITION BY userId) then 0
         else (rating - MIN(rating) OVER (PARTITION BY userId))/(MAX(rating) OVER (PARTITION BY userId) - MIN(rating) OVER (PARTITION BY userId))
       end normed_rating,
       AVG(rating) OVER (PARTITION BY userId) avg_rating
  FROM ratings
 ORDER BY userId, movieId
 LIMIT 30;
</pre>

![WindowFunctions](../img/hw3_1.png)

### ETL

ETL - процесс выгрузки данных, обработки и их дальнейшеней загрузки. В рамках домашней работы нужно проделать все три этапа

#### Extract

В директории data присутствует файл с ключевыми словами по фильмам:

<pre>
ls ./raw_data | grep keywords
</pre>

![Files_Keywords](../img/hw3_2.png)

Наша задача - загрузить это файл в Postgres. 

1. Напишите команду создания таблички keywords у неё должно быть 2 поля - id(числовой) и tags (текстовое).
2. Напишите команду копирования данных из файла в созданную вами таблицу
3. Проверьте, что в таблице есть записи
<pre>
#!/usr/bin/env bash
#Filename: hw3_keywords_import.sh

echo "Предварительно удаляем таблицу keywords при ее наличии..."

psql -U postgres -c "DROP TABLE IF EXISTS keywords"

echo "1. Создаем таблицу keywords..."

psql -U postgres -c "CREATE TABLE keywords ( id bigint, tags varchar(32000) );"

echo "2. Копирование данных из файла в созданную таблицу"

psql -U postgres -c "\copy keywords FROM '/usr/local/share/netology/raw_data/keywords.csv' DELIMITER ',' CSV HEADER"

echo "3. Проверяем, что в таблице есть записи"

psql -U postgres -c "SELECT COUNT(*) FROM keywords;"
</pre>

![Extract script](../img/hw3_3a.png)

![Extract](../img/hw3_3.png)

#### Transform

Мы загрузили данные в табличку, теперь нужно их преобразовать для дальнейшего использования. Мы ходитм узнать, какие теги у фильмов, которые сильно нравятся пользователям.

- Сформируйте запрос (назовём его ЗАПРОС1) к таблице ratings, в котором будут 2 поля
-- movieId
-- avg_rating - средний рейтинг, который ставят этому контенту пользователи
В выборку должны попасть те фильмы, которым поставили оценки более чем 50 пользователей
Список должен быть отсортирован по убыванию по полю avg_rating и по возрастанию по полю movieId
Из этой выборки оставить первое 150 элементов

Теперь мы хотим добавить к выборке хороших фильмов с высоким рейтингом информацию о тегах. Воспользуемся Common Table Expressions. Для этого нужно написать ЗАПРОС2, который присоединяет к выборке таблицу keywords

<pre>
-- transform by CTE
WITH top_rated as (SELECT movieId, AVG(rating) avg_rating
                     FROM ratings
                    GROUP BY movieId
                   HAVING COUNT(distinct userid) > 50
                    ORDER BY avg_rating DESC, movieId ASC
                    LIMIT 150
                  )
SELECT t.movieId, t.avg_rating, k.tags
  FROM top_rated t
  JOIN keywords k on t.movieId = k.id
 ORDER BY avg_rating DESC, movieId ASC;
</pre>

![Transform](../img/hw3_4.png)

#### Load

Мы обогатили выборку популярного контента внешними данными о тегах. Теперь мы можем сохранить эту информацию в таблицу для дальнейшего использования

Сохраним нашу выборку в новую таблицу top_rated_tags. Для этого мы модифицируем ЗАПРОС2 - вместо простого SELECT сделаем SELECT INTO.

Назовём всю эту конструкцию ЗАПРОС3
<pre>
-- ЗАПРОС3. Load into table TOP_RATED_TAGS
WITH top_rated as (SELECT movieId, AVG(rating) avg_rating
                     FROM ratings
                    GROUP BY movieId
                   HAVING COUNT(distinct userid) > 50
                    ORDER BY avg_rating DESC, movieId ASC
                    LIMIT 150
                  )
SELECT t.movieId, k.tags top_rated_tags
  INTO top_rated_tags
  FROM top_rated t
  JOIN keywords k on t.movieId = k.id
 ORDER BY avg_rating DESC, movieId ASC;
</pre>

![Load](../img/hw3_5.png)

![SELECT * FROM top_rated_tags;](../img/hw3_6.png)

Теперь можно выгрузить таблицу в текстовый файл - пример см. в лекции.

Внимание: Поля в текстовом файле нужно разделить при помощи табуляции ( символ E`\t`).

<pre>
#!/usr/bin/env bash
#Filename: hw3_keywords_export.sh

echo "Выгружаем таблицу в текстовый файл с разделителями - табуляцией..."

psql -U postgres -c "\copy (select * from top_rated_tags) to '/usr/local/share/netology/raw_data/top_rated_tags.tsv' with delimiter as E'\t'"
</pre>

![EXPORT](../img/hw3_7.png)

![Head of exported file](../img/hw3_7a.png)